In [261]:
pd.isna(False)

False

In [263]:
import pandas as pd  # module for working with data sets (need for working inner classes and modules)
import traceback
import datetime
import subprocess
import warnings
import numpy as np
import os  # module for working with operating system catalog structure
import openpyxl  # module for working with Excel files
import time  # module for working with date and time
import pyodbc  # module for working with databases
import win32com.client  # Module for generating MS access data base
import pyxlsb

from datetime import datetime  # Module for working with date and time data
from win32com.client import Dispatch  # Module for generating MS access data base
from tkinter.filedialog import askopenfilename  # Module for open file with win gui

# Ignoring pandas version errors
warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))

# show an "Open" dialog box and return the path to the selected file
filename_comp = askopenfilename(title="Select file for compare", filetypes=[("Excel Files", "*.xlsx"), ("Excel Binary Workbook", "*.xlsb")])
database_root = askopenfilename(title="Select database", filetypes=[('*.mdb', '*.accdb')]).replace('/', '\\')
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    fr'DBQ={database_root};'
    )

table = input("Input table's name : ")
with pyodbc.connect(conn_str) as conn:
    query = f'''SELECT * FROM {table}'''
    new_df = pd.read_sql(query, conn)

# Function for changing code 
def changing_code(df):
    date_expected, date_release = df['Код KKS документа'], df['Код KKS документа_new']
    if  not pd.isna(date_expected) and not pd.isna(date_release):
        return f'Смена кода с <{date_expected}> на <{date_release}>'
    else:
        return date_expected

# Function for changing name
def changing_name(df):
    date_expected, date_release = df['Наименование объекта/комплекта РД'], df['Наименование объекта/комплекта РД_new']
    if  not pd.isna(date_expected) and not pd.isna(date_release) and date_expected != date_release:
        return f'Смена наименования с <{date_expected}> на <{date_release}>'
    else:
        return date_expected

# Function for converting date
def convert_date(row):
    if row in ['в производстве', 'В производстве', None]:
        return row
    else:
        if dayfirst is True:
            return pd.to_datetime(row, dayfirst=dayfirst).date()
        else: 
            return pd.to_datetime(row, format='%Y/%m/%d').date()
    
#Function for changing status
def change_data(df, col_1, col_2):
    if ~pd.isna(df[col_1]):
        return df[col_2]
    else:
        return df[col_1]

# Function for changing_developer
def change_developer(df):
    if ~pd.isna(df['Разработчики РД (актуальные)']):
        return df['Разработчик РД']
    else:
        return df['Разработчики РД (актуальные)']

# Function for changing status
def change_status(df):
    if ~pd.isna(df['Статус текущей ревизии_new']):
        return df['Статус РД в 1С']
    else:
        return df['Статус текущей ревизии_new']

#Function for changing data
def changing_data(df):
    if (df[f'{col}'] == df[f'{col}_new']) or (pd.isna(df[f'{col}']) and pd.isna(df[f'{col}_new'])):
        return None
    else:
        return f'Смена {col.lower()} с <{df[f"{col}"]}> на <{df[f"{col}_new"]}>'

#Columns for changing dataframe

convert_columns = ['Дата выпуска РД по договору подрядчика', 
                   'Дата выпуска РД по графику с Заказчиком', 
                   'Дата статуса Заказчика', 
                   'Ожидаемая дата выдачи РД в производство', 
                   'Дата выпуска РД по договору подрядчика_new',
                   'Дата выпуска РД по графику с Заказчиком_new',
                   'Дата статуса Заказчика_new',
                   'Ожидаемая дата выдачи РД в производство_new']

clmns = ['Пакет РД', 'Статус Заказчика', 'Текущая ревизия', 'Статус текущей ревизии', 
           'Дата выпуска РД по договору подрядчика', 'Дата выпуска РД по графику с Заказчиком', 
           'Дата статуса Заказчика', 'Ожидаемая дата выдачи РД в производство']

inf_columns = ['Наименование объекта/комплекта РД',
               'Коды работ по выпуску РД', 
               'Пакет РД', 
               'Код KKS документа',
               'Статус Заказчика', 
               'Текущая ревизия', 
               'Статус текущей ревизии',
               'Дата выпуска РД по договору подрядчика',
               'Дата выпуска РД по графику с Заказчиком',
               'Дата статуса Заказчика',
               'Ожидаемая дата выдачи РД в производство', 
               'Письма',
               'Источник информации', 
               'Разработчики РД (актуальные)', 
               'Статус РД в 1С']

changed_cols = ['Коды работ по выпуску РД',
        'Наименование объекта/комплекта РД',
       'Код KKS документа',
       'Пакет РД',
       'Статус Заказчика', 
       'Текущая ревизия', 
       'Статус текущей ревизии',
       'Дата выпуска РД по договору подрядчика',
       'Дата выпуска РД по графику с Заказчиком', 'Дата статуса Заказчика',
       'Ожидаемая дата выдачи РД в производство',
       'Разработчики РД (актуальные)']

base_columns = ['Система',
                'Наименование объекта/комплекта РД',
                'Коды работ по выпуску РД',
                'Тип',
                'Пакет РД',	'Код KKS документа',
                'Статус Заказчика',
                'Текущая ревизия',
                'Статус текущей ревизии',
                'Дата выпуска РД по договору подрядчика',
                'Дата выпуска РД по графику с Заказчиком',
                'Дата статуса Заказчика',	
                'Ожидаемая дата выдачи РД в производство',	
                'Письма',	
                'Источник информации',	
                'Разработчики РД (актуальные)',	
                'Объект',	
                'WBS',	
                'КС',	
                'Примечания']

# read Excel files with current and new data
print('Read excel files with current and new data')
if '.xlsb' in filename_comp:
    with pyxlsb.open_workbook(filename_comp) as wb:
        with wb.get_sheet(1) as sheet:
            data = []
            for row in sheet.rows():
                data.append([item.v for item in row])
    base_df = pd.DataFrame(data[1:], columns=data[0])
else: 
    base_df = pd.read_excel(filename_comp)
new_df.columns = base_columns

#  Clear the data in both dataframe
print('Clear the empty rows in both dataframes')
base_df = base_df.dropna(subset=['Коды работ по выпуску РД'])
base_df['Разработчики РД (актуальные)'] = base_df.apply(change_developer, axis = 1)

# Removing unnecessary data
print('Clear the unnecessary data in base dataframe')
base_df = base_df.loc[~base_df['Коды работ по выпуску РД'].str.contains('.C.')]
base_df = base_df.loc[~base_df['Код KKS документа'].isin(['.KZ.', '.EK.', '.TZ.', '.KM.', '.GR.'])]

#  Making copy of original dataframes
print('Making copy of original dataframes')
base_df_copy = base_df.copy()
new_df_copy = new_df.copy()

#  Merging two dataframes dddd
print('Merging two dataframes')
missed_code = (new_df.merge(base_df,
                           how='left',
                           on=['Коды работ по выпуску РД'],
                           suffixes=['', '_new'], 
                           indicator=True))

m_df_1 = (pd.merge(new_df_copy, base_df_copy,
                           how='left',
                           on=['Коды работ по выпуску РД', 'Код KKS документа'],
                           suffixes=['', '_new'], 
                           indicator=True))
tmp_df = m_df_1[m_df_1['_merge'] == 'left_only'][new_df_copy.columns]
m_df_1 = m_df_1[m_df_1['_merge'] == 'both']
m_df_1['Наименование объекта/комплекта РД'] = m_df_1.apply(lambda row: changing_name(row), axis = 1)

m_df_2 = (tmp_df.merge(base_df_copy,
                           how='left',
                           on=['Коды работ по выпуску РД', 'Наименование объекта/комплекта РД'],
                           suffixes=['', '_new'],
                           indicator=True))
m_df_2 = m_df_2[m_df_2['_merge'] == 'both']
m_df_2['Код KKS документа'] = m_df_2.apply(lambda row: changing_code(row), axis = 1)

# Converting types
print('Converting types')
m_df_1['Статус текущей ревизии_new'] = m_df_1.apply(change_status, axis = 1)
m_df_2['Статус текущей ревизии_new'] = m_df_2.apply(change_status, axis = 1)

for col in convert_columns:
    if 'new' in col:
        m_df_1[col] = m_df_1[col].apply(convert_date)
        m_df_2[col] = m_df_2[col].apply(convert_date)
    else:
        dayfirst = True
        m_df_1[col] = m_df_1[col].apply(convert_date)
        m_df_2[col] = m_df_2[col].apply(convert_date)

for col in clmns:
    m_df_1[f'{col}'] = m_df_1.apply(changing_data)
    m_df_2[f'{col}'] = m_df_2.apply(changing_data)

Read excel files with current and new data
Clear the empty rows in both dataframes
Clear the unnecessary data in base dataframe
Making copy of original dataframes
Merging two dataframes
Converting types


KeyError: 'Пакет РД'

In [281]:
def changing_data(df):
    if (df[f'{col}'] == df[f'{col}_new']) or (pd.isna(df[f'{col}']) and pd.isna(df[f'{col}_new'])):
        return None
    else:
        return f'Смена {col.lower()} с <{df[f"{col}"]}> на <{df[f"{col}_new"]}>'

# for col in clmns:
#     print(col)
col = 'Текущая ревизия'
m_df_1['Текущая ревизия'] = m_df_1.apply(lambda df: None if (df[f'{col}'] == df[f'{col}_new']) or (pd.isna(df[f'{col}']) and pd.isna(df[f'{col}_new'])) else f'Смена {col.lower()} с <{df[f"{col}"]}> на <{df[f"{col}_new"]}>')

KeyError: 'Текущая ревизия'

In [274]:
m_df_1

,Система,Наименование объекта/комплекта РД,Коды работ по выпуску РД,Тип,Пакет РД,Код KKS документа,Статус Заказчика,Текущая ревизия,Статус текущей ревизии,Дата выпуска РД по договору подрядчика,...,Количество исходных файлов,Код работы СМР,Наименование работы СМР,id_obj,Объект_new,Группа,WBS_new,Вид работ,Статус РД в 1С,_merge
0,ИМ,"Внешние присоединения кабелей к НКУ 0,4 кВ в ч...",10UQA.E.B00.D-1020,Схема подключений,Смена пакет рд с <Смена пакет рд с <None> на <...,AKU.1324.10UQA.0.AK.TB0002,Смена статус заказчика с <(rev.C01)> на <На ВК...,C01 (есть только в 1С),None,2021-12-24,...,None,10UQA.E.B00.C-1175,"10UQA. Внешние присоединения кабелей к НКУ 0,4...",5090.0,10UQA - Основная насосная станция,Пусковой комплекс №1,EM - Силовое электрооборудование,Электротехника,На согласовании AN,both
1,ИМ,Вторичные соединения в части электродвигателей...,10UQA.E.B00.D-1110,Схема электрическая,Смена пакет рд с <Смена пакет рд с <None> на <...,AKU.1324.10UQA.0.AK.TB0001,Смена статус заказчика с <(rev.C01)> на <Выдан...,C01 (есть только в 1С),None,2021-12-24,...,None,10UQA.E.B00.C-1230,10UQA. Вторичные соединения элементов питания ...,5099.0,10UQA - Основная насосная станция,Пусковой комплекс №1,EM - Силовое электрооборудование,Электротехника,Передан в производство,both
2,СКУ ЭЧ ЭБ,"Вторичные соединения элементов питания РУСН-0,...",10UQA.E.B00.D-1130,Схема подключений,Смена пакет рд с <Смена пакет рд с <None> на <...,AKU.1324.10UQA.0.AE.TB0004,Смена статус заказчика с <(rev.C01)> на <ВК - ...,C01 (есть только в 1С),None,2021-12-24,...,None,10UQA.E.B00.C-1240,10UQA. Вторичные соединения элементов питания ...,5101.0,10UQA - Основная насосная станция,Пусковой комплекс №1,EM - Силовое электрооборудование,Электротехника,На корректировке у разработчика,both
77,Технология,Резервуар для сбора нефтесодержащих стоков. Те...,01UEH.T.TTT.D-1030,Схемы ТП,Смена пакет рд с <Смена пакет рд с <Смена паке...,None,None,None,None,None,...,None,01UEH.C.U00.C-1030,01UEH. Технологические решения,107454.0,01UEH - Приемное сооружение для дизельного топ...,Пусковой комплекс №1,01UEH - Приемное сооружение для дизельного топ...,Технологические решения,None,both
78,Технология,Резервуар для сбора нефтесодержащих стоков. Те...,00UEJ.T.TTT.D-1030,Схемы ТП,Смена пакет рд с <Смена пакет рд с <Смена паке...,None,None,None,None,None,...,None,00UEJ.C.H00.C-1050,00UEJ. Технологические решения,107451.0,00UEJ - Склад дизельного топлива,Пусковой комплекс №1,T - Технологические системы,Технологические решения,None,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3660,СВБУ,(12UBP). Структурная схема СВБУ,10UJA.J.JJJ.D-1230,Принципиальная схема,Смена пакет рд с <Смена пакет рд с <None> на <...,AKU.0120.10UJA.0.AT.FA0003,None,C02,DD signed FOR CONSTRUCTION / Комплект РД подпи...,None,...,None,10UJA.E.B00.C-6695,10UJA. (11UBP). Внешние присоединения для обор...,60175.0,10UJA - Реакторное здание,Пусковой комплекс №1,"AE - Релейная защита, автоматика и управление",Электротехника,Передан в производство,both
3661,СВБУ,(11UBP). Структурная схема СВБУ,10UJA.J.JJJ.D-1270,Принципиальная схема,Смена пакет рд с <Смена пакет рд с <None> на <...,AKU.0120.10UJA.0.AT.FA0001,None,C02,DD signed FOR CONSTRUCTION / Комплект РД подпи...,None,...,None,10UJA.E.B00.C-6630,10UJA. (11UBP). Внешние присоединения для обор...,72038.0,10UJA - Реакторное здание,Пусковой комплекс №1,"AE - Релейная защита, автоматика и управление",Электротехника,Передан в производство,both
3761,СКУ ПЗ,Задание заводу на шкафы управления огнезадержи...,10UMA.J.JJJ.D-1160,ЗЗИ/ИДИ,Смена пакет рд с <Смена пакет рд с <None> на <...,AKU.0130.10UMA.0.AP.EC0001,None,C01,Comments from T2 to sub-supplier / Замечания о...,2022-03-01,...,None,10UMA.E.JJJ.C-3370,10UMA. Задание заводу на шкафы управления прот...,25542.0,10UMA - Здание турбины,Пусковой комплекс №1,PS - Пожарная сигнализация,Пожарная безопасность,На корректировке у разработчика,both
3762,АСУ ТП,Задание заводу на программное обеспечение 